In [424]:
import pandas as pd
import numpy as np

In [425]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [426]:
data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
test_data.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)


In [427]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=0)

In [428]:
for train_index, test_index in sss.split(data,data['Survived']):
    train_data = data.iloc[train_index]
    valuation_data = data.iloc[test_index]

In [429]:
X_train = train_data.drop(['Survived'],axis=1)
y_train = train_data['Survived']

X_valuation = valuation_data.drop(['Survived'],axis=1)
y_valuation = valuation_data['Survived']


In [430]:
X_train = pd.get_dummies(data=X_train,columns=['Sex'])
X_valuation = pd.get_dummies(data=X_valuation,columns=['Sex'])
test_data = pd.get_dummies(data=test_data,columns=['Sex'])

X_train = pd.get_dummies(data=X_train,columns=['Embarked'])
X_valuation = pd.get_dummies(data=X_valuation,columns=['Embarked'])
test_data = pd.get_dummies(data=test_data,columns=['Embarked'])

In [431]:
X_train = X_train.astype(float)
X_valuation = X_valuation.astype(float)
test_data = test_data.astype(float)

In [432]:
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_valuation['Age'].fillna(X_valuation['Age'].mean(),inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

/var/folders/8_/kb8645j910n7lcgmfs8c0d0m0000gn/T/ipykernel_25961/3688605551.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
/var/folders/8_/kb8645j910n7lcgmfs8c0d0m0000gn/T/ipykernel_25961/3688605551.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting va

In [433]:
from sklearn.preprocessing import StandardScaler

In [434]:
sc = StandardScaler()

In [435]:
X_train = sc.fit_transform(X_train)
X_valuation = sc.transform(X_valuation)
test_data = sc.transform(test_data)

In [436]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [437]:
reg = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()
svm = SVC()
models = [reg,rf,xgb,svm]

In [438]:
best_accuracy = -1
best_model = None
for model in models:
    model.fit(X_train,y_train)
    y_val = model.predict(X_valuation)
    acc = accuracy_score(y_valuation,y_val)
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model
print(f"The best model is {best_model} with accuracy of {best_accuracy}")    

The best model is SVC() with accuracy of 0.8100558659217877


In [439]:
y_pred = model.predict(test_data)

In [440]:
result = pd.DataFrame(
    {
        "PassengerId":list(range(892,892+len(y_pred))),
        "Survived" : y_pred
    }
    )
result.to_csv("submission.csv",index=False)